In [1]:
import os
from together import Together
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
import os
from together import Together

client = Together(api_key=os.environ.get('TOGETHER_API_KEY'))

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3-70B-Instruct-Turbo",
    messages=[{"role": "user", "content": "What are some fun things to do in New York?"}],
    max_tokens=512,
    temperature=0.7,
    top_p=0.7,
    top_k=50,
    repetition_penalty=1,
    stop=["<|eot_id|>"],
    stream=True
)
print(response.choices[0].message.content)

UnboundLocalError: local variable 'result' referenced before assignment

In [ ]:
response.choices

AttributeError: 'generator' object has no attribute 'choices'

In [ ]:

client = Together(api_key=os.environ.get("TOGETHER_API_KEY"))

stream = client.chat.completions.create(
  model="meta-llama/Meta-Llama-3-70B-Instruct-Turbo",
  messages=[{"role": "user", "content": "What are some fun things to do in New York?"}],
  stream=False,
)

for chunk in stream:
  print(chunk.choices[0].delta.content or "", end="", flush=True)


AttributeError: 'tuple' object has no attribute 'choices'

In [ ]:
stream

ChatCompletionResponse(id='8a6528d7d836536c-CAI', object=<ObjectType.ChatCompletion: 'chat.completion'>, created=1721501670, model='meta-llama/Meta-Llama-3-70B-Instruct-Turbo', choices=[ChatCompletionChoicesData(index=0, logprobs=None, finish_reason=<FinishReason.EOS: 'eos'>, message=ChatCompletionMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content="The city that never sleeps! New York City is a vibrant and exciting destination that offers something for everyone. Here are some fun things to do in New York:\n\n**Iconic Landmarks and Attractions:**\n\n1. **Statue of Liberty and Ellis Island**: Take a ferry to Liberty Island to see the iconic statue up close and visit the Ellis Island Immigration Museum.\n2. **Central Park**: A tranquil oasis in the middle of Manhattan, perfect for a stroll, picnic, or bike ride.\n3. **Empire State Building**: For a panoramic view of the city, head to the observation deck of this iconic skyscraper.\n4. **Grand Central Terminal**: Admire the stunnin